In [11]:
import os
import pandas as pd
import rpy2.robjects as ro
from rpy2.robjects.packages import importr
from rpy2.robjects import pandas2ri

import missingno as msno

from sklearn.ensemble import GradientBoostingClassifier

from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, KFold, RandomizedSearchCV

from sklearn.metrics import auc, roc_curve, roc_auc_score, classification_report, confusion_matrix, accuracy_score 

from sklearn.utils.class_weight import compute_sample_weight



import numpy as np
import matplotlib.pyplot as plt
import sage 
import shap

import xgboost as xgb

from hyperopt import STATUS_OK, Trials, fmin, hp, tpe

In [12]:
os.chdir('/home/jlm217/rds/rds-mrc-bsu-csoP2nj6Y6Y/mimah/stopah/stopah/data') 

pd.set_option('display.max_columns', None)  

ro.r['load']('STOPAH_ForSolon.RData')

def R_dataset(x):
    with (ro.default_converter + pandas2ri.converter).context():
        stopah = ro.conversion.get_conversion().rpy2py(ro.r[x])
    return stopah

stopah = R_dataset('stopah')

R[write to console]: Warning:
R[write to console]:  namespace ‘pROC’ is not available and has been replaced
by .GlobalEnv when processing object ‘roc.objs’



In [13]:
selected = ['D28_DTH','D90_DTH','Prednisolone']

baseline = ['Gender','Baseline_sepsis','Baseline_GIB',
'Age.at.randomisation..calc.','Weight','Max.grams.of.alcohol.drunk.per.day..calc.','Hepatic.Encephalopathy...Merged',
'Temperature...Merged','Pulse...Merged','Systolic.BP...Merged','Diastolic.BP...Merged','MAP','Hb...Merged','Platelets...Merged',
'WBC...Merged','Neutrophils...Merged','INR...Merged.clinical.and.calc','Bilirubin.Merged','ALT...Merged','ALP...Merged',
'Albumin...Merged','Sodium...Merged','Potassium...Merged','Urea...Merged','Creatinine...Merged','NLR_0','bDNA',
'Ferritin_ngml','Iron_mumoll','Transferrin','TSAT','PNPLA3_Add','PNPLA3_Rec','HPCT_NG'] 

sevenday = ['Hepatic.Encephalopathy...Treatment.Day.7..',
'Day.7.infection','Gastrointestinal.Bleed.since.the.last.visit.Gastrointestinal.bleed...and.Choose..Treatment.Day.7..',
'Temperature..Treatment.Day.7..','Pulse..Treatment.Day.7..',
'Systolic.BP..Treatment.Day.7..','Diastolic.BP..Treatment.Day.7..','MAP..Treatment.Day.7',
'Hb..1.decimal.point..Haematology..Treatment.Day.7..','Platelets.day.7','WBC.day.7',
'Neutrophils.day.7','INR.clinical.and.calc.day.7','Bilirubin.day.7','ALT.day.7',
'ALP.day.7','Albumin.day.7','Sodium.day.7','Potassium.day.7','Urea.day.7','Creatinine.day.7']

#reduce data set to target, treatment and therapy

stopah = stopah[sevenday+['Prednisolone','D28_DTH']]

stopah.reset_index(drop=True, inplace=True)

In [14]:
df = stopah.dropna()

df0 = df[df['Prednisolone']==0].drop(['Prednisolone'],axis=1)

df = df[df['Prednisolone']==1].drop(['Prednisolone'],axis=1)

X, y = df.drop('D28_DTH', axis=1), df[['D28_DTH']]

X0, y0 = df0.drop('D28_DTH', axis=1), df0[['D28_DTH']]

In [15]:
cats = ['Hepatic.Encephalopathy...Treatment.Day.7..', 'Day.7.infection',
              'Gastrointestinal.Bleed.since.the.last.visit.Gastrointestinal.bleed...and.Choose..Treatment.Day.7..']

#for col in cats:
 #   X[col] = X[col].astype('category')

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

X0_train, X0_test, y0_train, y0_test = train_test_split(X0, y0, random_state=1)

In [17]:
#Train

#Treated

clf = GradientBoostingClassifier(n_estimators=1000, 
                                 learning_rate=0.1,
                                 max_depth=7,
                                 #min_child_weight = 1,
                                 random_state=0).fit(X_train, y_train)

#Untreated

clf0= GradientBoostingClassifier(n_estimators=1000, 
                                 learning_rate=0.1,
                                 max_depth=7, 
                                #in_child_weight = 1,
                                 random_state=0).fit(X0_train, y0_train)


A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().


### Untreated vs treated training and test performance

In [18]:
print('Untreated')
print()
print('XGB Score:'+str(clf0.score(X0_test, y0_test)))

y0_predict = clf0.predict_proba(X0_test)

auc_score0 = roc_auc_score(y0_test, y0_predict[:,1])
print('AUC Score:',(auc_score0)*100)

print()
print('Treated')
print()

print('XGB Score:'+str(clf.score(X_test, y_test)))

y_predict = clf.predict_proba(X_test)

auc_score = roc_auc_score(y_test, y_predict[:,1])
print('AUC Score:',(auc_score)*100)

Untreated

XGB Score:0.7790697674418605
AUC Score: 74.70238095238095

Treated

XGB Score:0.8133333333333334
AUC Score: 81.8888888888889


In [ ]:
space={'max_depth': hp.quniform("max_depth", 3, 18, 1),
        'gamma': hp.uniform ('gamma', 1,9),
        'reg_alpha' : hp.quniform('reg_alpha', 40,180,1),
        'reg_lambda' : hp.uniform('reg_lambda', 0,1),
        'colsample_bytree' : hp.uniform('colsample_bytree', 0.5,1),
        'min_child_weight' : hp.quniform('min_child_weight', 0, 10, 1),
        'n_estimators': 180,
        'seed': 0
    }

In [22]:
def objective(space):
    clf=xgb.XGBClassifier(
                    n_estimators =space['n_estimators'], max_depth = int(space['max_depth']), gamma = space['gamma'],
                    reg_alpha = int(space['reg_alpha']),min_child_weight=int(space['min_child_weight']),
                    colsample_bytree=int(space['colsample_bytree']))
    
    evaluation = [( X_train, y_train), ( X_test, y_test)]
    
    clf.fit(X_train, y_train,
            eval_set=evaluation, eval_metric="auc",
            early_stopping_rounds=10,verbose=False)
    

    pred = clf.predict(X_test)
    accuracy = accuracy_score(y_test, pred>0.5)
    print ("SCORE:", accuracy)
    return {'loss': -accuracy, 'status': STATUS_OK }


In [35]:
best_hyperparams

{'colsample_bytree': 0.7809296185634191,
 'gamma': 6.553897555295923,
 'max_depth': 8.0,
 'min_child_weight': 7.0,
 'reg_alpha': 79.0,
 'reg_lambda': 0.2617290944816928}

In [37]:
# Define the hyperparameter space
space = {
    'max_depth': hp.quniform('max_depth', 2, 8, 1),
    'learning_rate': hp.loguniform('learning_rate', -5, -2),
    'subsample': hp.uniform('subsample', 0.5, 1)
}

# Define the objective function to minimize
def objective(params):
    xgb_model = xgb.XGBClassifier(**params)
    xgb_model.fit(X_train, y_train)
    y_pred = xgb_model.predict(X_test)
    score = accuracy_score(y_test, y_pred)
    return {'loss': -score, 'status': STATUS_OK}

# Perform the optimization
best_params = fmin(objective, space, algo=tpe.suggest, max_evals=100)
print("Best set of hyperparameters: ", best_params)



  0%|          | 0/100 [00:00<?, ?trial/s, best loss=?]

job exception: Invalid Parameter format for max_depth expect int but value='7.0'



  0%|          | 0/100 [00:00<?, ?trial/s, best loss=?]


XGBoostError: Invalid Parameter format for max_depth expect int but value='7.0'